In [2]:
"""
Notebook for estimating the power spectral density 
of a signal streaming from a microphone in realtime

audio is captured using pyaudio
then displayed using matplotlib in a separate window using the tk backend

if you don't have pyaudio, then run

>>> pip install pyaudio

Adapted from: https://github.com/markjay4k/Audio-Spectrum-Analyzer-in-Python/tree/master

"""

import pyaudio

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import EngFormatter

from tkinter import TclError

# to display in separate Tk window
%matplotlib tk



In [3]:
# pyaudio class instance
p = pyaudio.PyAudio()

# configure the audio device (microphone)
FS = p.get_default_input_device_info()['defaultSampleRate']/2
CHUNK = 1024 * 2              # samples per frame
FORMAT = p.get_format_from_width(2, False)   # audio format (bytes per sample?)
CHANNELS = 1                  # single channel for microphone
RATE = int(FS)                # samples per second

In [4]:
# create a stream object to get data from microphone
stream = p.open(
    format=FORMAT,
    channels=CHANNELS,
    rate=RATE,
    input=True,
    output=False,
    frames_per_buffer=CHUNK
)


In [ ]:
## create matplotlib figure
fig = plt.figure(figsize=(15,7))

# x-axis variable for plotting
ff = (np.arange(CHUNK) / CHUNK - 0.5) * FS

# create a line object with initial date (off screen)
# `line_inst` shows the instantaneous PSD estimate (periodogram)
# `line_mean` shows the (exponentially weighted, see beta) average of periodograms 
line_inst, = plt.plot(ff, -200*np.ones_like(ff), 
                      '-', lw=2, label='Instantaneous')
line_mean, = plt.plot(ff, -200*np.ones_like(ff), 
                      '--', lw=4, label='Average')
plt.grid()

beta = 0.05

# format spectrum axes
formatter_ff = EngFormatter(unit='Hz')
plt.xlim(0, FS / 2)
plt.ylim(-60, 60)  # <<<------- these may need some tweaking
plt.gca().xaxis.set_major_formatter(formatter_ff)
plt.ylabel('Power Spectral Density (dB)')

leg = plt.legend()


In [ ]:
## real-time capture and processing
# Send an interrupt to stop
print('stream started')

# initialization prior to loop
frame_count = 0
ss_mean = np.zeros_like(ff)

while True:
    
    # binary data
    try:
        data = stream.read(CHUNK, exception_on_overflow=False)    
    except:
        pass
    
    data_np = np.frombuffer(data, dtype=np.int16)
    
    # compute FFT and update line
    ss_inst = np.fft.fftshift(np.abs(np.fft.fft(data_np*(1.0/CHUNK), norm='ortho'))**2)
    ss_mean = ss_mean*(1-beta) + ss_inst*beta

    line_inst.set_ydata(10*np.log10(ss_inst))
    line_mean.set_ydata(10*np.log10(ss_mean))
    
    # update figure canvas
    try:
        fig.canvas.draw_idle()
        fig.canvas.flush_events()
        frame_count += 1
        
    except (KeyboardInterrupt, TclError):
        print('stream stopped')
        break

stream.stop_stream()
stream.close()
p.terminate()

stream started


Exception in Tkinter callback
Traceback (most recent call last):
  File "/opt/local/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/tkinter/__init__.py", line 1892, in __call__
    return self.func(*args)
  File "/opt/local/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/tkinter/__init__.py", line 814, in callit
    func(*args)
  File "/opt/local/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/matplotlib/backends/_backend_tk.py", line 271, in idle_draw
    self.draw()
  File "/opt/local/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/matplotlib/backends/backend_tkagg.py", line 10, in draw
    super().draw()
  File "/opt/local/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/matplotlib/backends/backend_agg.py", line 387, in draw
    self.figure.draw(self.renderer)
  File "/opt/local/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/matplotlib/art